In [4]:
import numpy as np
np.random.seed(42)
import tensorflow as tf
tf.random.set_seed(42)
from keras.models import Sequential, load_model
from keras.layers import Dense, Activation
from keras.layers import LSTM, Dropout
from keras.layers import TimeDistributed
from keras.layers.core import Dense, Activation, Dropout, RepeatVector
from tensorflow.keras.optimizers import RMSprop
import matplotlib.pyplot as plt
import pickle
import sys
import heapq
import seaborn as sns
from pylab import rcParams

In [5]:
sns.set(style='whitegrid', palette='muted', font_scale=1.5)

rcParams['figure.figsize'] = 12, 5

path = '1661-0.txt'
text = open(path).read().lower()
print('corpus length:', len(text))

chars = sorted(list(set(text)))
char_indices = dict((c, i) for i, c in enumerate(chars))
indices_char = dict((i, c) for i, c in enumerate(chars))


corpus length: 581888


In [6]:
SEQUENCE_LENGTH = 40
step = 3
sentences = []
next_chars = []
for i in range(0, len(text) - SEQUENCE_LENGTH, step):
    sentences.append(text[i: i + SEQUENCE_LENGTH])
    next_chars.append(text[i + SEQUENCE_LENGTH])
print(len(sentences))

193950


In [7]:
X = np.zeros((len(sentences), SEQUENCE_LENGTH, len(chars)), dtype=np.bool)
y = np.zeros((len(sentences), len(chars)), dtype=np.bool)
for i, sentence in enumerate(sentences):
    for t, char in enumerate(sentence):
        X[i, t, char_indices[char]] = 1
    y[i, char_indices[next_chars[i]]] = 1


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: DeprecationWarning: `np.bool` is a deprecated alias for the builtin `bool`. To silence this warning, use `bool` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.bool_` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  """Entry point for launching an IPython kernel.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: DeprecationWarning: `np.bool` is a deprecated alias for the builtin `bool`. To silence this warning, use `bool` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.bool_` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  


In [8]:
model = Sequential()
model.add(LSTM(128, input_shape=(SEQUENCE_LENGTH, len(chars))))
model.add(Dense(len(chars)))
model.add(Activation('softmax'))

In [9]:
print("gere")

gere


In [10]:
optimizer = RMSprop(lr=0.01)
model.compile(loss='categorical_crossentropy', optimizer=optimizer, metrics=['accuracy'])
history = model.fit(X, y, validation_split=0.05, batch_size=128, epochs=20, shuffle=True).history

/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/rmsprop.py:130: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(RMSprop, self).__init__(name, **kwargs)


Epoch 1/20
1440/1440 [==============================] - 202s 138ms/step - loss: 1.9814 - accuracy: 0.4190 - val_loss: 2.1107 - val_accuracy: 0.4105
Epoch 2/20
1440/1440 [==============================] - 190s 132ms/step - loss: 1.6265 - accuracy: 0.5128 - val_loss: 2.0326 - val_accuracy: 0.4437
Epoch 3/20
1440/1440 [==============================] - 188s 131ms/step - loss: 1.5213 - accuracy: 0.5423 - val_loss: 2.0281 - val_accuracy: 0.4483
Epoch 4/20
1440/1440 [==============================] - 191s 132ms/step - loss: 1.4675 - accuracy: 0.5554 - val_loss: 2.0255 - val_accuracy: 0.4595
Epoch 5/20
1440/1440 [==============================] - 193s 134ms/step - loss: 1.4319 - accuracy: 0.5656 - val_loss: 2.0250 - val_accuracy: 0.4604
Epoch 6/20
1440/1440 [==============================] - 193s 134ms/step - loss: 1.4081 - accuracy: 0.5717 - val_loss: 2.0191 - val_accuracy: 0.4603
Epoch 7/20
1440/1440 [==============================] - 192s 134ms/step - loss: 1.3875 - accuracy: 0.5779 - val_

In [12]:
model.save('keras_model.h5')
pickle.dump(history, open("history.p", "wb"))

In [13]:
model = load_model('keras_model.h5')
history = pickle.load(open("history.p", "rb"))

In [14]:
def prepare_input(text):
    x = np.zeros((1, SEQUENCE_LENGTH, len(chars)))
    for t, char in enumerate(text):
        x[0, t, char_indices[char]] = 1.
        
    return x

In [15]:
def sample(preds, top_n=3):
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds)
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    
    return heapq.nlargest(top_n, range(len(preds)), preds.take)

In [16]:
def predict_completion(text):
    original_text = text
    generated = text
    completion = ''
    while True:
        x = prepare_input(text)
        preds = model.predict(x, verbose=0)[0]
        next_index = sample(preds, top_n=1)[0]
        next_char = indices_char[next_index]
        text = text[1:] + next_char
        completion += next_char
        
        if len(original_text + completion) + 2 > len(original_text) and next_char == ' ':
            return completion

In [29]:
def predict_completions(text, n=3):
    x = prepare_input(text)
 
    preds = model.predict(x, verbose=0)[0]
  
    next_indices = sample(preds, n)
    print(text)
    
    return [indices_char[idx] + predict_completion(text[1:] + indices_char[idx]) for idx in next_indices]



In [18]:
quotes = [
    "It is not a lack of love, but a lack of friendship that makes unhappy marriages.",
    "That which does not kill us makes us stronger.",
    "I'm not upset that you lied to me, I'm upset that from now on I can't believe you.",
    "And those who were seen dancing were thought to be insane by those who could not hear the music.",
    "It is hard enough to remember my opinions, without also remembering my reasons for them!"
]



In [30]:
for q in quotes:
    seq = q[:40].lower()
    print(seq)
    print(predict_completions(seq, 5))
    print()

it is not a lack of love, but a lack of 
it is not a lack of love, but a lack of 
['the ', 'strength ', 'a ', 'his ', 'bright ']

that which does not kill us makes us str
that which does not kill us makes us str
['ength ', 'ick ', 'ange ', 'ong ', 'uck ']

i'm not upset that you lied to me, i'm u
i'm not upset that you lied to me, i'm u
['nderstand ', 'se ', 'pon ', 'mplay ', 'gh ']

and those who were seen dancing were tho
and those who were seen dancing were tho
['ught ', 'se ', 're ', 'nd ', 'med ']

it is hard enough to remember my opinion
it is hard enough to remember my opinion
[' of ', '. ', 'al ', 's ', ', ']



In [54]:
complete_my_sentence= ["who does a bit of simple cooking and keeps the place clean."]

In [55]:
for q in complete_my_sentence:
    seq = q[:40].lower()
    print(seq)
    print(predict_completions(seq, 10))
    print()

who does a bit of simple cooking and kee
who does a bit of simple cooking and kee
['ped ', 'n ', 'med ', 'dver ', ' and ', 'bver, ', 'cked ', 'k ', 'gled ', '\nthe ']

